In [1]:
import pandas as pd
import datetime
from pandas import Series,DataFrame

# RUBO2

## 读取RUBO2(1).xlsx中Packlist-sheetname

rubo2 = pd.read_excel('RUBO2(1).xlsx',sheet_name='Packlist',index_col=None,header=0,skiprows=0)

## rubo2_Packlist中筛选出需要的字段信息

# 将rubo2中字段重命名
rubo2.rename(columns={'Unnamed: 0':'产品代码',  'Packlist':'客户运单号','Unnamed: 7':'目的省份',\
                            'WMS SHIP DATE':'发货时间','CARRIER CODE':'运输类型','CTNS':'预计箱数',\
                            'Unit':'预计发货','SHIP TO':'客户代码'},inplace=True)

# 选择rubo2中需要读取的字段
rubo2_col_list = ['客户运单号','产品代码','目的省份','ST NAME','发货时间','预计箱数','客户代码','Window','Address1','Address2','Address3']

# basedata.xlsx中的Sheet'CLC预报'和'客户信息表'

## 读取Sheet'CLC预报

basedata_CLC = pd.read_excel('basedata.xlsx',sheet_name='CLC预报',index_col=None,header=0,skiprows=0)

### Sheet'CLC预报'中筛选出所需字段

basedata_CLC.rename(columns={'发货单号':'客户运单号'},inplace=True)

# '客户运单号'=‘发货单号’。‘BU’代表‘产品代码’，rubo2表中有这里没有提取
basedata_CLC_col_list = ['总单号','客户运单号','出发城市','目的城市',\
                         '卸货地址','箱数','件数','陆运/铁运/空运','在途时间',\
                         '预计到达时间','客户简称','客户名称','联系方式','联系人手机']

## 读取Sheet'客户信息表'

basedata_Customersinfo = pd.read_excel('basedata.xlsx',sheet_name='客户信息表',index_col=None,header=0,skiprows=0)

### Sheet'客户信息表'中筛选出所需字段

# 将basedata_Customersinfo中字段‘-’重命名为'客户代码'
basedata_Customersinfo.rename(columns={'-':'客户代码'},inplace=True)

# 选择basedata_Customersinfo中需要读取的字段
basedata_Customersinfo_col_list = ['客户代码','详细地址','城市','省份','电话1','手机预报','电话2','电话-2011','联系人']

# 转换字段‘客户代码’数据的数据类型为str
basedata_Customersinfo.客户代码=basedata_Customersinfo.客户代码.astype(str)

# 调整basedata_Customersinfo中字段内容
basedata_Customersinfo['联系人电话'] = list(map(lambda x,y,z:str(x)+str(y)+str(z),\
                                           basedata_Customersinfo['电话1'],\
                                           basedata_Customersinfo['电话2'],\
                                           basedata_Customersinfo['电话-2011'] ))

basedata_Customersinfo['联系人手机'] = basedata_Customersinfo['手机预报']

# 选择basedata_Customersinfo中需要读取的字段
basedata_Customersinfo_col_list = ['客户代码','详细地址','城市','省份','联系人','联系人电话','联系人手机']

# 将数据中值为nan得数据填充为空字符串‘’
basedata_Customersinfo.fillna('',inplace=True)

# 根据字段'客户运单号'合并表RUBO2_Packlist和表basedata_CLC预报

# 转换字段‘客户运单号’数据的数据类型为str
rubo2.客户运单号 = rubo2.客户运单号.astype(str)
basedata_CLC.客户运单号 = basedata_CLC.客户运单号.astype(str)

merged_packlist_CLC = pd.merge(rubo2.loc[:,rubo2_col_list],basedata_CLC.loc[:,basedata_CLC_col_list],on='客户运单号')

# 将merged_packlist_CLC_col_list的所有字段名保存到列表
merged_packlist_CLC_col_list = merged_packlist_CLC.columns.tolist()

# 转换字段‘客户代码’数据的数据类型为str
merged_packlist_CLC.客户代码=merged_packlist_CLC.客户代码.astype(str)
basedata_Customersinfo.客户代码= basedata_Customersinfo.客户代码.astype(str)

# 根据字段'客户代码'合并表merged_packlist_CLC和表basedata_客户信息表

merged_allinfo = pd.merge(merged_packlist_CLC.loc[:,merged_packlist_CLC_col_list],basedata_Customersinfo.loc[:,basedata_Customersinfo_col_list],on='客户代码')

# 添加rubo2_packlist.客户代码不在basedata_Customersinfo.客户代码中的字段rubo2_packlist.客户代码信息到merged_allinfo

# rubo2_packlist.客户代码在basedata_Customersinfo.客户代码中记为lm
lm = []
lb = []
for m in merged_packlist_CLC.客户代码:
    for b in basedata_Customersinfo.客户代码:
        if m == b:
            lm.append(m)
        else:
            lb.append(b)

# rubo2_packlist.客户代码不在database_customerinfo.客户代码中
list(set(merged_packlist_CLC.客户代码)^set(lm))
# 载rubo2_packlist.客户代码中提取不在basedata_Customersinfo.客户代码中的每条记录
for e in list(set(merged_packlist_CLC.客户代码)^set(lm)):
    merged_packlist_CLC[merged_packlist_CLC['客户代码']== e]
    merged_allinfo = pd.concat([pd.DataFrame(merged_allinfo),merged_packlist_CLC[merged_packlist_CLC['客户代码']== e]],\
                               sort=True,ignore_index=True)

# 调整字段数据的格式和表中字段的排列顺序

# 将数据中值为nan得数据填充为空字符串‘’
merged_allinfo.fillna('',inplace=True)

merged_allinfo['发货时间'] = Series(map(lambda x:'{}'.format('20'+str(x)),merged_allinfo['发货时间']))

merged_allinfo['客户运单号'] = Series(map(lambda x:'{:09}'.format(int(x)),merged_allinfo['客户运单号']))

merged_allinfo['产品代码'] = Series(map(lambda x:'APP' if x=='AP' else x,merged_allinfo['产品代码']))

# 联系人（个人客户直接ST NAME,为公司：取客户信息的配置表)
merged_allinfo.loc[merged_allinfo[merged_allinfo['客户名称'] =='个人客户'].index.tolist(),['联系人']]=\
merged_allinfo[merged_allinfo['客户名称'] =='个人客户']['ST NAME']

# 联系人手机（为公司：配置表的预留手机，为个人用户时：ST NAME或SHIP_TO_PHONE_NBR)
merged_allinfo.loc[merged_allinfo[merged_allinfo['客户名称'] =='个人客户'].index.tolist(),['联系人手机']]=\
merged_allinfo[merged_allinfo['客户名称'] =='个人客户']['ST NAME']

# 卸货地点：公司客户取客户信息表的详细地址，个人客户或为空运时：Address2+Address3+Address1
merged_allinfo.loc[merged_allinfo[merged_allinfo['客户名称'] =='个人客户'].index.tolist(),['卸货地址']]=\
 list(map(lambda x,y,z:str(x)+'/'+str(y)+'/'+str(z),\
                                           merged_allinfo[merged_allinfo['客户名称'] =='个人客户']['Address2'],\
                                           merged_allinfo[merged_allinfo['客户名称'] =='个人客户']['Address3'],\
                                           merged_allinfo[merged_allinfo['客户名称'] =='个人客户']['Address1'] ))

basedata_Customersinfo['联系人电话'] = list(map(lambda x,y,z:str(x)+str(y)+str(z),\
                                           basedata_Customersinfo['电话1'],\
                                           basedata_Customersinfo['电话2'],\
                                           basedata_Customersinfo['电话-2011'] ))

datetime.datetime.now().strftime("%Y%m")
merged_allinfo['结算月份'] = datetime.datetime.now().strftime("%Y%m")

merged_allinfo.预计到达时间 = list(map(lambda x:x.replace('-','/'),merged_allinfo.预计到达时间.astype(str)))
merged_allinfo.总单号 = list(map(lambda x:x.replace('-','/'),merged_allinfo.总单号.astype(str)))

# 调整merged_allinfo写入到excel文件的字段顺序
merged_allinfo = merged_allinfo.loc[:,['客户运单号','客户代码','客户简称', '客户名称','联系人','联系人电话','联系人手机',\
                      '卸货地址', '箱数', '件数', '在途时间','发货时间','预计到达时间','陆运/铁运/空运',\
                      '出发城市', '目的城市','目的省份','产品代码','总单号','结算月份']]

# 写到Excel文件

merged_allinfo.to_excel('mergedallinfo.xlsx')